# P01 - 11: Importing and exporting network data

**April 23 2020**   
*Ingo Scholtes*  

Now that we have understood the basics of networks in `pathpy`, we need a way to read and write network data. In the following, we show how we can easily import and export networks from and to common file formats, pandas dataframes or databases. For this, we first import `pathpy` as well as `pandas` and `sqlite3`:

In [1]:
import pathpy as pp

import pandas as pd
import sqlite3

The simplest way to store network data is in terms of an adjacency list, i.e. a simple text file where each line contains the uids of source and target nodes of an edge, separated by a special character. This widely used format is the default file format of `pathpy` (and many other network analysis packages). We can use the `write_csv` function in the `io` module to save a network instance in this format. In the generation of the toy network below, we also demonstrate how we can add multiple nodes and edges at once based on a sequence of node uid tuples. The call to `add_edges` below generates three nodes and two edges:

In [2]:
n = pp.Network()
n.add_edges(('a', 'b'), ('a', 'c'))
print(n)

Uid:			0x143c805ae48
Type:			Network
Directed:		True
Multi-Edges:		False
Number of nodes:	3
Number of edges:	2


To store this network in a CSV file, we call:

In [3]:
pp.io.write_csv(n, 'network.csv')

If you inspect this file, you will find that it contains the source and target node uids of all edges, as well as the edge uids. By default a comma separator is used, but we can easily change this using the `sep` parameter of the function. The `io` functions of `pathpy` are based on `pandas` data frames, and saving a network to a csv file will internally first create a pandas data frame, then storing the data frame to a csv file. If we want to manually use a dataframe representation of the network, we can use the `to_dataframe` function in the `io` module, e.g. calling:

In [4]:
df = pp.io.to_dataframe(n)
df

,v,w,uid
0,a,c,0x143c8078160
1,a,b,0x143c805af60


Just as we can create a data frame from a network, we can also create a network from a data frame. This data frame must minimally include two columns 'v' and 'w' that store the uids of the nodes that are connected by edges. If no edge uids exist, new uids will be created by default. If a uid column is passed, those uids will be used.

In [5]:
df = pd.DataFrame(columns=['v', 'w'])
df.loc[0] = ['a', 'b']
df.loc[1] = ['b', 'c']
n = pp.io.from_dataframe(df, uid='pandasNetwork')
print(n)
print(n.nodes.uids)
print(n.edges.uids)

Uid:			pandasNetwork
Type:			Network
Directed:		True
Multi-Edges:		False
Number of nodes:	3
Number of edges:	2
{'a', 'b', 'c'}
{'0x143c80e4438', '0x143c80e42e8'}


To directly load a network from a csv file, we can use the `read_csv` function in the `io` module. To read the network that we saved before, we can write:

In [6]:
n = pp.io.read_csv('network.csv')
print(n)

Uid:			0x143c80dae48
Type:			Network
Directed:		True
Multi-Edges:		False
Number of nodes:	3
Number of edges:	2


Since the `io` functions are internally based on pandas data frames, we would get the same network (with a different uid though) if we did the following:

In [7]:
df = pd.read_csv('network.csv')
n = pp.io.from_dataframe(df)
print(n)

Uid:			0x143c805acc0
Type:			Network
Directed:		True
Multi-Edges:		False
Number of nodes:	3
Number of edges:	2


The above call will generate a network with a new uid and no attributes. if we want to assign attributes or a custom uid to the newly generated network, we can simply pass those attributes to the `from_csv` function:

In [8]:
n = pp.io.read_csv(filename='network.csv', uid='csvnetwork')
print(n)

Uid:			csvnetwork
Type:			Network
Directed:		True
Multi-Edges:		False
Number of nodes:	3
Number of edges:	2


All of the functions above also work with edges that have arbitrary attributes. If only part of the edges have a given attribute, a NaN value will be automatically assigned for other edges. Let's create a small example where this is the case:

In [9]:
n = pp.Network(directed=False)
n.add_edge('a', 'b', weight=2.0)
n.add_edge('a', 'c', type='friendship')
print(n)

Uid:			0x143c80e44a8
Type:			Network
Directed:		False
Multi-Edges:		False
Number of nodes:	3
Number of edges:	2


 To export the edges of a network to a data frame we write:

In [10]:
df = pp.io.to_dataframe(n)
df

,v,w,uid,type,weight
0,a,c,0x143c805aeb8,friendship,NaN
1,a,b,0x143c805af28,NaN,2.0


In [11]:
pp.io.write_csv(n, 'network_attributes.csv')

We can now read a network from this csv file and obtain the same data as before:

In [12]:
n = pp.io.read_csv('network_attributes.csv')

print(n)

for e in n.edges:
    print(e)
    print(e.attributes)

Uid:			0x143c805aba8
Type:			Network
Directed:		True
Multi-Edges:		False
Number of nodes:	3
Number of edges:	2
Uid:		0x143c805aeb8
Type:		Edge
Source node:	Node a
Target node:	Node c
{'type': 'friendship', 'weight': nan}
Uid:		0x143c805af28
Type:		Edge
Source node:	Node a
Target node:	Node b
{'type': nan, 'weight': 2.0}


## Storing network data in databases

While the adjacency list file format is maximally simple and widely available, it also has disadvantages. First, it can be quite cumbersome to deal with a collection of external data files where each file defines its own seprator character. Moreover, we don't have a simple way to e.g. filter the data or make advanced queries without manipulating text files. 

For scenarios where we want to analyse data from heterogeneous sources, it is often simpler to store such data in a lightweight database like `SQLite`. `SQLite` is a simple but powerful file-based database management system that is supported in `python` without the need to install any external module, let alone a client-server database system. `SQLite` supports most of SQL and it is implemented in a lightweight C library. Moreover, there are simple and lightweight OpenSource tools to manipulate and manage `SQLite` database files, e.g. the [DB-Browser for SQLite](http://sqlitebrowser.org) or the SQLite plugin for VSCode.

To simplify the analysis of network data, `pathpy` can directly load network from SQL databases. For this, we can call the function `read_sql` function in the `io` module. It allows us to pass an open connection to an SQLite database file, which we obtain using the `connect` function of sqlite3. We can further pass an arbitrary SQL query. The columns of this query will be used to generate edges (with attributes), just like in the case of a csv file or data frame. 

In [13]:
con = sqlite3.connect('networks.db')
n = pp.io.read_sql(con=con, sql='SELECT source, target FROM lotr', directed=True)
print(n)

[08-17 17:33:06: ERROR] The edge "0x143c85427b8" already exists in the Network


KeyError: 

If we instead want to read all columns from a given table (without giving an open connection or an SQL query) we can simply write:

In [14]:
n = pp.io.read_sql(filename='networks.db', table='gentoo', directed=True)
print(n)

Uid:			0x143c857c198
Type:			Network
Directed:		True
Multi-Edges:		False
Number of nodes:	403
Number of edges:	513


If we read a whole table from an SQL database, the `read_sql` function will automatically retrieve all edge attributes. In our database, the table `highschool`, which captures contact traces of highschool students, contains a weight attribute that stores the number of times two students have been in close proximity.

In [15]:
n = pp.io.read_sql(filename='networks.db', table='highschool', directed=False)
e = list(n.edges.uids)[0]
print(n.edges[e])
print(n.edges[e].attributes)

[08-17 17:34:43: ERROR] The edge "0x143c8820438" already exists in the Network


KeyError: 

Storing a Network in an SQL database is just as easy. Let's first create a network where edges have attributes:

In [16]:
n = pp.Network(directed=False)
n.add_edge('a', 'b', weight=2.0)
n.add_edge('a', 'c', type='friendship')
print(n)

Uid:			0x143c85962b0
Type:			Network
Directed:		False
Multi-Edges:		False
Number of nodes:	3
Number of edges:	2


To store this in a new table in an SQLite database file, we call the `write_sql` function. If the database file and table do not exist, a new database file or table is created. if they exist, we can use the `if_exists` parameter to specify what should be done:

In [17]:
pp.io.write_sql(n, filename='networks.db', table='test_network', if_exists='replace')